# **LLM Project to Build and Fine Tune a Large Language Model**

In today's data-driven world, the ability to process and generate natural language text at scale has become a transformative force across industries. Large Language Models (LLMs) represent a cutting-edge advancement in natural language processing, enabling businesses to extract valuable insights, automate tasks, and enhance user experiences. By harnessing the power of LLMs, organizations can improve customer service, automate content creation, and gain a competitive edge in the digital landscape.

This project builds the foundation for Large Language Models by diving deep into the details of their inner workings. Moreover, It shows how to optimize their use through prompt engineering and fine-tuning techniques such as LoRA. 

Prompt engineering techniques involve crafting specific instructions or queries given to the language model to influence its output will be introduced to guide LLMs in generating desired responses through zero-shot, one-shot, and few-shot inferences.

Fine-tuning entails training a pre-trained language model on a specific task or dataset to adapt it for a particular application. It explores full fine-tuning and Parameter Efficient Fine Tuning (PEFT), a technique that optimizes the fine-tuning process by focusing on a subset of the model's parameters, making it more resource-efficient.

The project also involves the application of Retrieval Augmented Generation (RAG) using OpenAI's GPT-3.5 Turbo, resulting in the development of a chatbot for online shopping for knowledge grounding. Knowledge grounding with Retrieval Augmented Generation (RAG) is implemented to mitigate hallucinations and provide trustworthy and reliable responses. This is achieved by incorporating information from external sources to validate and support the generated text.

For example, in the context of an e-commerce chatbot using RAG, knowledge grounding ensures that product information, availability, and prices are sourced from a trusted database or e-commerce platform. This prevents the chatbot from generating inaccurate or fictional details and instead provides responses based on real-world data.


![image](https://images.pexels.com/photos/18069697/pexels-photo-18069697/free-photo-of-an-artist-s-illustration-of-artificial-intelligence-ai-this-illustration-depicts-language-models-which-generate-text-it-was-created-by-wes-cockx-as-part-of-the-visualising-ai-project-l.png?auto=compress&cs=tinysrgb&w=1260&h=750&dpr=1)

## **Learning Outcomes**

* Understand Large Language Models (LLMs) and how they work.
* Gain practical experience in implementing generative AI projects.
* Understand fundamental NLP concepts like RNNs, Transformers, and Attention Mechanism.
* Explore tokenization, embeddings, and internal workings of Transformers.
* Generate text and summarize dialogues using LLMs.
* Learn optimization techniques like Prompt Engineering, Fine-Tuning, and PEFT.
* Apply Prompt Engineering techniques for better responses.
* Fine-tune LLMs for improved performance on tasks.
* Evaluate model performance using the ROUGE metric.
* Understand RLHF for improved model output.
* Implement Retrieval Augmented Generation (RAG) for knowledge grounding.
* Build a chatbot application for online shopping.


In [ ]:
# python- 3.8.10 
# !pip install --upgrade pip
# !pip install transformers
# !pip install datasets --quiet
# !pip install torchdata
# !pip install torch
# !pip install streamlit
# !pip install openai
# !pip install langchain
# !pip install unstructured
# !pip install sentence-transformers
# !pip install chromadb
# !pip install evaluate==0.4.0
# !pip install rouge_score==0.1.2
# !pip install loralib==0.1.1
# !pip install peft==0.3.0

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import torch
import evaluate
import time
import pandas as pd
import numpy as np
from datasets import load_dataset
from transformers import (AutoModelForSeq2SeqLM, AutoModelForCausalLM, 
                          AutoTokenizer, GenerationConfig, TrainingArguments, Trainer)
from transformers import AutoTokenizer
from transformers import GenerationConfig


## **Refresher**

## **Recurrent Neural Networks**

RNN were created because there were a few issues in the feed-forward neural network:

Cannot handle sequential data
Considers only the current input
Cannot memorize previous inputs
The solution to these issues is the RNN. An RNN can handle sequential data, accepting the current input data, and previously received inputs. RNNs can memorize previous inputs due to their internal memory.

RNN works on the principle of saving the output of a particular layer and feeding this back to the input in order to predict the output of the layer.

Below is how you can convert a Feed-Forward Neural Network into a Recurrent Neural Network

<img src="images/rnn.png" 
     align="center" 
     width="700" />


The nodes in different layers of the neural network are compressed to form a single layer of recurrent neural networks. A, B, and C are the parameters of the network.

<img src="images/rnn_animation.gif" 
     align="center" 
     width="450" />


The four commonly used types of Recurrent Neural Networks are:

**One-to-One**: The simplest type of RNN is One-to-One, which allows a single input and a single output. It has fixed input and output sizes and acts as a traditional neural network. The One-to-One application can be found in Image Classification.

**One-to-Many**: One-to-Many is a type of RNN that gives multiple outputs when given a single input. It takes a fixed input size and gives a sequence of data outputs. Its applications can be found in Music Generation and Image Captioning.

**Many-to-One**: Many-to-One is used when a single output is required from multiple input units or a sequence of them. It takes a sequence of inputs to display a fixed output. Sentiment Analysis is a common example of this type of Recurrent Neural Network.

**Many-to-Many**: Many-to-Many is used to generate a sequence of output data from a sequence of input units.

This type of RNN is further divided into ﻿the following two subcategories:

    1. Equal Unit Size: In this case, the number of both the input and output units is the same. A common application can be found in Name-Entity Recognition.
    2. Unequal Unit Size: In this case, inputs and outputs have different numbers of units. Its application can be found in Machine Translation.



<img src="images/types_rnn.png" 
     align="center" 
     width="750" />

## **LSTM**

Now, even though RNNs are quite powerful, they suffer from Vanishing gradient problem which hinders them from using long term information, like they are good for storing memory 3-4 instances of past iterations but larger number of instances don't provide good results so we don't just use regular RNNs. Instead, we use a better variation of RNNs: Long Short Term Networks(LSTM).

**What is Vanishing Gradient problem?**

Vanishing gradient problem is a difficulty found in training artificial neural networks with gradient-based learning methods and backpropagation. In such methods, each of the neural network's weights receives an update proportional to the partial derivative of the error function with respect to the current weight in each iteration of training. The problem is that in some cases, the gradient will be vanishingly small, effectively preventing the weight from changing its value. In the worst case, this may completely stop the neural network from further training. As one example of the problem cause, traditional activation functions such as the hyperbolic tangent function have gradients in the range (0, 1), and backpropagation computes gradients by the chain rule. This has the effect of multiplying n of these small numbers to compute gradients of the "front" layers in an n-layer network, meaning that the gradient (error signal) decreases exponentially with n while the front layers train very slowly.

<img src="images/decay.png" 
     align="center" 
     width="750" />


**Fixing the Vanishing/Exploding Gradient with LSTMs**

Long short-term memory (LSTM) units (or blocks) are a building unit for layers of a recurrent neural network (RNN). A RNN composed of LSTM units is often called an LSTM network. A common LSTM unit is composed of a cell, an input gate, an output gate and a forget gate. The cell is responsible for "remembering" values over arbitrary time intervals; hence the word "memory" in LSTM. Each of the three gates can be thought of as a "conventional" artificial neuron, as in a multi-layer (or feedforward) neural network: that is, they compute an activation (using an activation function) of a weighted sum. Intuitively, they can be thought as regulators of the flow of values that goes through the connections of the LSTM; hence the denotation "gate". There are connections between these gates and the cell.

The expression long short-term refers to the fact that LSTM is a model for the short-term memory which can last for a long period of time. An LSTM is well-suited to classify, process and predict time series given time lags of unknown size and duration between important events. LSTMs were developed to deal with the exploding and vanishing gradient problem when training traditional RNNs.


<img src="images/lstm.png" 
     align="center" 
     width="750" />

More on LSTMs: https://medium.com/deep-math-machine-learning-ai/chapter-10-1-deepnlp-lstm-long-short-term-memory-networks-with-math-21477f8e4235



## **Attention Mechanism**

In a typical neural network, all parts of the input sequence are treated equally. However, in many tasks, certain parts of the input may be more relevant than others. For example, in a sentence, the word "dog" might be more important than "the" for understanding the meaning.

The attention mechanism addresses this by allowing the model to focus on specific parts of the input when processing it. It does this by assigning weights or scores to different elements of the input sequence. These weights reflect how much attention the model should pay to each element. The weighted sum of the input elements then becomes the basis for making predictions.

To know more refer to the project [Build a Text Classification Model with Attention Mechanism NLP](https://www.projectpro.io/project-use-case/multi-class-text-classification-model-with-attention-mechanism-in-nlp)

## **Transformers**

Transformers are a type of neural network architecture that heavily rely on attention mechanisms. They were introduced in the paper ["Attention is All You Need"](https://arxiv.org/pdf/1706.03762.pdf).

Quoting from the paper: 

*The Transformer is the first transduction model relying entirely on self-attention to compute representations of its input and output without using sequence-aligned RNNs or convolution.*

Transformers are a class of powerful neural network architectures that have revolutionized natural language processing (NLP) and various other sequential data tasks. What sets Transformers apart is their unique attention mechanism, which allows them to process input data in parallel rather than sequentially. This means they can consider the relationships between all elements in a sequence simultaneously, making them exceptionally efficient. The attention mechanism works by assigning weights to different parts of the input, highlighting the most relevant information. Transformers are composed of multiple layers, each containing a multi-head self-attention sub-layer and a feedforward neural network sub-layer. These layers are stacked to form the core of the model. 

Refer to the project [NLP Project for Multi Class Text Classification using BERT Model](https://www.projectpro.io/project-use-case/nlp-project-for-multi-class-text-classification-using-bert) to implement transformers from scratch.

Read more https://towardsdatascience.com/transformers-89034557de14

## **Tokenizers**

Tokenization is a fundamental step in Natural Language Processing (NLP) that involves breaking down a text into smaller units, known as tokens. Tokens can be words, subwords, or characters, depending on the level of granularity required for the task at hand. In this tutorial, we'll explore various aspects of tokenization, including different tokenization techniques, libraries, and considerations for tokenizing text data.

## **Embeddings**

Word embeddings are a crucial concept in Natural Language Processing (NLP). They represent words as numerical vectors in a continuous vector space. This tutorial will provide a conceptual understanding of word embeddings, their significance, and different methods for generating them.

Word embeddings represent words in a continuous vector space, where similar words are positioned closer to each other. This enables algorithms to understand the context and relationships between words.

Unlike one-hot encoding (a sparse representation), where each word is represented as a binary vector, word embeddings are dense vectors with continuous values. This allows for more nuanced representation of word meanings.

Word embeddings capture semantic relationships between words. Words with similar meanings will have similar vector representations.


To know more refer to the projects [Word2Vec and FastText Word Embedding with Gensim in Python](https://www.projectpro.io/project-use-case/word-embedding-with-word2vec-and-fasttext) and [Skip Gram Model Python Implementation for Word Embeddings](https://www.projectpro.io/project-use-case/skip-gram-model-word-embeddings-python).

## **Text Generation**

In recent years, there has been an increasing interest in open-ended language generation thanks to the rise of large transformer-based language models trained on millions of webpages, including OpenAI's ChatGPT and Meta's LLaMA. The results on conditioned open-ended language generation are impressive, having shown to generalize to new tasks, handle code, or take non-text data as input. Besides the improved transformer architecture and massive unsupervised training data, better decoding methods have also played an important role.

Currently most prominent decoding methods, mainly Greedy search, Beam search, and Sampling.

In [3]:
DEVICE = 'cpu'

In [4]:
torch_device = torch.device(DEVICE)

In [5]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

# add the EOS token as PAD token to avoid warnings
model = AutoModelForCausalLM.from_pretrained("gpt2", pad_token_id=tokenizer.eos_token_id).to(torch_device)

## Greedy Search

Greedy search is the simplest decoding method. It selects the word with the highest probability as its next word: 

![title](images/greedy_search.png)

Starting from the word "The", the algorithm greedily chooses the next word of highest probability "nice" and so on, so that the final generated word sequence is ("The", "nice", "woman")
having an overall probability of 0.5×0.4=0.2




In [6]:
# encode context the generation is conditioned on
model_inputs = tokenizer('I enjoy walking with my cute dog', return_tensors='pt').to(torch_device)

# generate 40 new tokens
greedy_output = model.generate(**model_inputs, max_new_tokens=40)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(greedy_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with my dog. I'm not sure if I'll ever be able to walk with my dog.

I'm not sure


We have generated our first short text with GPT2!

The generated words following the context are reasonable, but the model quickly start repeating itself! This is a very common problem in language generation in general and seems to be even more so in greedy and beam search. The major drawback of greedy search though is that it misses high probability words hidden behind low probability word as can be seen in the sketch above.

The word "has" with its high conditional probability of 0.9 hidden behind the word "dog", which has only the second-highest conditional probability, so that greedy search misses the word sequence "The", "dog", "has".

## **Beam Search**

Beam search reduces the risk of missing hidden high probability word sequences by keeping the most likely num_beams of hypotheses at each time step and eventually choosing the hypothesis that has the overall highest probability. Let's illustrate with num_beams=2:

![title](images/beam_search.png)

At time step 1, besides the most likely hypothesis ("The", "nice"), beam search also keeps track of the second most likely one ("The", "dog"). At time step 2, beam search finds that the word sequence ("The", "dog", "has") with 0.36 higher probability than ("the", "nice", "woman") which has 0.2. Great, it has found the most likely word sequence in the example.

Beam search will always find an output sequence with higher probability than greedy search, but its not guaranteed to find the most likely output

In [7]:
# activate beam search and early_stopping
beam_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=2,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I don't think I'll ever be able to walk with my dog again.

I'm not sure if I'll ever be able to walk with my dog again, but I don


While the result is arguably more fluent, the output still includes repetitions of the same word sequences. One of the available remedies is to introduce n-grams. The most common n-grams penalty makes sure that no n-gram appears twice by manually setting the probability of next words that could create an already seen n-gram to 0.

In [8]:
# set no_repeat_ngram_size to 2
beam_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    no_repeat_ngram_size=2,
    early_stopping=True
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(beam_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to


We can see that the repetition does not appear anymore. Nevertheless, n-gram penalties have to be used with care. An article generated about the city New York should not use a 2-gram penalty or otherwise, the name of the city would only appear once in the whole text!

Another important feature about beam search is that we can compare the top beams after generation and choose the generated beam that fits our purpose best.

In transformers, we simply set the parameter num_return_sequences to the number of highest scoring beams that should be returned. Make sure though that num_return_sequences <= num_beams!

In [9]:
# set return_num_sequences > 1
beam_outputs = model.generate(
    **model_inputs,
    max_new_tokens=40,
    num_beams=5,
    no_repeat_ngram_size=2,
    num_return_sequences=5,
    early_stopping=True
)

# now we have 3 output sequences
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  print("{}: {}".format(i, tokenizer.decode(beam_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time for me to
1: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with her again.

I've been thinking about this for a while now, and I think it's time for me to
2: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's a good idea to
3: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's time to take a
4: I enjoy walking with my cute dog, but I'm not sure if I'll ever be able to walk with him again.

I've been thinking about this for a while now, and I think it's a good idea

As can be seen, the five beam hypotheses are only marginally different to each other - which should not be too surprising when using only 5 beams.

## **Sampling**

In its most basic form, sampling means randomly picking the next word according to its conditional probability distribution

![title](images/sampling_search.png)

It becomes obvious that language generation using sampling is not deterministic anymore. The word ("car") is sampled from the conditioned probability distribution P(w|"The"), followed by sampling ("drives") from P(w|"The", "car")

In transformers, we set do_sample=True and deactivate Top-K sampling (more on this later) via top_k=0. In the following, we will fix the random seed for illustration purposes. Feel free to change the set_seed argument to obtain different results, or to remove it for non-determinism.

In [10]:
# set seed to reproduce results. Feel free to change the seed though to get different results
from transformers import set_seed
set_seed(42)

# activate sampling and deactivate top_k by setting top_k sampling to 0
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog but what I love about being a dog cat person is being a pet being with people who can treat you. I feel happy to be such a pet person and get to meet so many people. I


Interesting! The text seems alright - but when taking a closer look, it is not very coherent and doesn't sound like it was written by a human. That is the big problem when sampling word sequences: The models often generate incoherent gibberish

A trick is to make the distribution sharper (increasing the likelihood of high probability words and decreasing the likelihood of low probability words) by lowering the so called temperature of the softmax

<img src="images/sampling_search_with_temp.png" width="800" height="400">

The conditional next word distribution of step T=1 becomes much sharper leaving almost no chance for word ("car") to be selected.

In [11]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# use temperature to decrease the sensitivity to low probability candidates
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=0,
    temperature=0.6,
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog but I also love the fact that my cat is not a dog. She is a good, loving dog. I do not like to be held back by other dogs but I think that I have to


There were less weird n-grams and the output is a bit more coherent now. However, while applying temperature can make a distribution less random, in its limit, when setting temperature -> 0, temperature scaled sampling becomes equal to greedy decoding and will suffer from the same problems as before.

## **Top-K Sampling**

In Top-K sampling, the K most likely next words are filtered and the probability mass is redistributed among only those K next words. GPT2 adopted this sampling scheme, which was one of the reasons for its success in story generation.

<img src="images/top_k_sampling.png" width="1200" height="600">

In [12]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# set top_k to 50
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=35,
    do_sample=True,
    top_k=50
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog but what I love about being a dog is I see a beautiful pet being cared for – I love having the opportunity to see her every day so I feel very privileged to have


Not bad at all! The text is arguably the most human-sounding text so far. One concern though with Top-K sampling is that it does not dynamically adapt the number of words that are filtered from the next word probability distribution. This can be problematic as some words might be sampled from a very sharp distribution (distribution on the right in the graph above), whereas others from a much more flat distribution (distribution on the left in the graph above).

## **Top-p (nucleus) sampling**

Instead of sampling only from the most likely K words, in Top-p sampling chooses from the smallest possible set of words whose cumulative probability exceeds the probability p. The probability mass is then redistributed among this set of words. This way, the size of the set of words (a.k.a the number of words in the set) can dynamically increase and decrease according to the next word's probability distribution. Ok, that was very wordy, let's visualize.

<img src="images/top_p_sampling.png" width="1200" height="600">

Having set p = 0.92, Top-p sampling picks the minimum number of words to exceed together p = 92% of the probability mass. In the first example, this included the 9 most likely words, whereas it only has to pick the top 3 words in the second example to exceed 92%.

In [13]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# set top_k to 50
sample_output = model.generate(
    **model_inputs,
    max_new_tokens=35,
    do_sample=True,
    top_p=0.92,
    top_k=0
)

print("Output:\n" + 100 * '-')
print(tokenizer.decode(sample_output[0], skip_special_tokens=True))

Output:
----------------------------------------------------------------------------------------------------
I enjoy walking with my cute dog but what I love about being a dog cat person is being a pet being with people who can treat you. I feel happy to be such a pet person and get to meet


While in theory, Top-p seems more elegant than Top-K, both methods work well in practice. Top-p can also be used in combination with Top-K, which can avoid very low ranked words while allowing for some dynamic selection.

In [14]:
# set seed to reproduce results. Feel free to change the seed though to get different results
set_seed(42)

# set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
sample_outputs = model.generate(
    **model_inputs,
    max_new_tokens=40,
    do_sample=True,
    top_k=50,
    top_p=0.95,
    num_return_sequences=3,
)

print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  print("{}: {}".format(i, tokenizer.decode(sample_output, skip_special_tokens=True)))

Output:
----------------------------------------------------------------------------------------------------
0: I enjoy walking with my cute dog but sometimes I get nervous when she is around. I've been told that with her alone, she will usually wander off and then try to chase me. It's nice to know that I have this
1: I enjoy walking with my cute dog. I think she is the same one I like to walk with my dog, I think she is about as girly as my first dog. I hope we can find an apartment for her when we
2: I enjoy walking with my cute dog, but there's so much to say about him that I am going to miss it all. He has been so supportive and even had my number in his bag.

I hope I can say


# **Dialogue Summarization**

In this use case, we will be generating a summary of a dialogue with the pre-trained Large Language Model (LLM) FLAN-T5 from Hugging face.

Let's upload some simple dialogues from the DialogSum Hugging Face dataset. This dataset contains 10.000+ dialogues with the corresponding manually labeled summaries and topics.

In [15]:
torch_device = torch.device(DEVICE)

In [16]:
huggingface_dataset_name = 'knkarthick/dialogsum'
dataset = load_dataset(huggingface_dataset_name)

Extracting data files: 100%|██████████| 3/3 [00:00<00:00, 85.79it/s]
Generating train split: 12460 examples [00:00, 26184.78 examples/s]
Generating validation split: 500 examples [00:00, 10931.04 examples/s]
Generating test split: 1500 examples [00:00, 24702.31 examples/s]


In [17]:
example_indices = [40, 200]

In [18]:
dash_line = "-".join('' for x in range(100))

for i, index in enumerate(example_indices):
    print(dash_line)
    print('Example ', i+1)
    print(dash_line)
    print('Input Dialogue: ')
    print(dataset['test'][index]['dialogue'])
    print(dash_line)
    print('Baseline Human Summary: ')
    print(dataset['test'][index]['summary'])
    print(dash_line)
    print()
    

---------------------------------------------------------------------------------------------------
Example  1
---------------------------------------------------------------------------------------------------
Input Dialogue: 
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------

---------------------------------------------------------------------------------------------------
E

## **FLAN-T5 Model**

<img src="images/flan2_architecture.jpg" width="1000" height="500">

#### Flan-PaLM 540B achieves state-of-the-art performance on several benchmarks, such as 75.2% on five-shot MMLU. We also publicly release Flan-T5 checkpoints,1 which achieve strong few-shot performance even compared to much larger models, such as PaLM 62B. Overall, instruction finetuning is a general method for improving the performance and usability of pretrained language models

In [19]:
model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

## Training

<img src="images/flan_t5_tasks.png" width="900" height="450">

#### These models are based on pretrained T5 (Raffel et al., 2020) and fine-tuned with instructions for better zero-shot and few-shot performance. There is one fine-tuned Flan model per T5 model size.

## Inference

In [20]:
sentence = "What time is it, Tom?"

In [21]:
sentence_encoded = tokenizer(sentence, return_tensors='pt')

sentence_decoded = tokenizer.decode(sentence_encoded["input_ids"][0], skip_special_tokens=True)

print(f'ENCODED SENTENCE:\n {sentence_encoded["input_ids"][0]}')
print(f'DECODED SENTENCE: {sentence_decoded}')

ENCODED SENTENCE:
 tensor([ 363,   97,   19,   34,    6, 3059,   58,    1])
DECODED SENTENCE: What time is it, Tom?


In [22]:
def summarize_dialogues(example_indices, dataset, prompt = "%s"):
    for i, index in enumerate(example_indices):
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
    
        input = prompt % (dialogue)
        
        inputs = tokenizer(input, return_tensors='pt')
        pred = model.generate(inputs["input_ids"], max_new_tokens=50)[0]
        output = tokenizer.decode(pred, skip_special_tokens=True)
    
        print(dash_line)
        print(f'Example {i+1}')
        print(dash_line)
        print(f'Input Prompt: \n {dialogue}')
        print(dash_line)
        print(f'Baseline Human Summary: \n {summary}')
        print(dash_line)
        print(f'Model Generation: \n{output}\n')

In [23]:
summarize_dialogues(example_indices, dataset)

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Prompt: 
 #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation: 
Person1: It's ten to nine.

--------------------------------------------------------

### Zero Shot Inference with an Instruction Prompt

In [24]:
prompt = f'Summarize the following conversation. \n%s\nSummary:'
print(prompt)

Summarize the following conversation. 
%s
Summary:


In [25]:
summarize_dialogues(example_indices, dataset, prompt)

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Prompt: 
 #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation: 
The train is about to leave.

------------------------------------------------------

In [26]:
prompt = f'Dialogue: \n%s\n\nWhat Happened?'
print(prompt)

Dialogue: 
%s

What Happened?


In [27]:
summarize_dialogues(example_indices, dataset, prompt)

---------------------------------------------------------------------------------------------------
Example 1
---------------------------------------------------------------------------------------------------
Input Prompt: 
 #Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.
---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
 #Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.
---------------------------------------------------------------------------------------------------
Model Generation: 
Tom is late.

----------------------------------------------------------------------

# One Shot Inference

In [28]:
def make_prompt(example_indices_full, example_index_to_summarize):
    prompt = ''
    for index in example_indices_full:
        dialogue = dataset['test'][index]['dialogue']
        summary = dataset['test'][index]['summary']
        
        prompt += f"""Dialogue:\n{dialogue}\n\nWhat was going on?\n{summary}\n\n\n"""
        dialogue = dataset['test'][example_index_to_summarize]['dialogue']

    prompt += f'Dialogue:\n{dialogue}\n\nWhat was going on?'
    return prompt

In [29]:
example_indices_full = [40]
example_index_to_summarize = 200

one_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(one_shot_prompt)

Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


Dialogue:
#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a

In [30]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(one_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(inputs["input_ids"], max_new_tokens=50)[0], skip_special_tokens=True
)

print(dash_line)
print(f'Baseline Human Summary: \n{summary}\n')
print(dash_line)
print(f'Model Generation - One Shot:\n{output}')

---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
Model Generation - One Shot:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to add a CD-ROM drive.


# Few Shot Inference

In [31]:
example_indices_full = [40, 80, 120]
example_index_to_summarize = 200

few_shot_prompt = make_prompt(example_indices_full, example_index_to_summarize)

print(few_shot_prompt)

Dialogue:
#Person1#: What time is it, Tom?
#Person2#: Just a minute. It's ten to nine by my watch.
#Person1#: Is it? I had no idea it was so late. I must be off now.
#Person2#: What's the hurry?
#Person1#: I must catch the nine-thirty train.
#Person2#: You've plenty of time yet. The railway station is very close. It won't take more than twenty minutes to get there.

What was going on?
#Person1# is in a hurry to catch a train. Tom tells #Person1# there is plenty of time.


Dialogue:
#Person1#: May, do you mind helping me prepare for the picnic?
#Person2#: Sure. Have you checked the weather report?
#Person1#: Yes. It says it will be sunny all day. No sign of rain at all. This is your father's favorite sausage. Sandwiches for you and Daniel.
#Person2#: No, thanks Mom. I'd like some toast and chicken wings.
#Person1#: Okay. Please take some fruit salad and crackers for me.
#Person2#: Done. Oh, don't forget to take napkins disposable plates, cups and picnic blanket.
#Person1#: All set. May,

In [32]:
summary = dataset['test'][example_index_to_summarize]['summary']

inputs = tokenizer(few_shot_prompt, return_tensors='pt')
output = tokenizer.decode(
    model.generate(inputs["input_ids"], max_new_tokens=50)[0], skip_special_tokens=True
)

print(dash_line)
print(f'Baseline Human Summary: \n{summary}\n')
print(dash_line)
print(f'Model Generation - Few Shot:\n{output}')

Token indices sequence length is longer than the specified maximum sequence length for this model (818 > 512). Running this sequence through the model will result in indexing errors


---------------------------------------------------------------------------------------------------
Baseline Human Summary: 
#Person1# teaches #Person2# how to upgrade software and hardware in #Person2#'s system.

---------------------------------------------------------------------------------------------------
Model Generation - Few Shot:
#Person1 wants to upgrade his system. #Person2 wants to add a painting program to his software. #Person1 wants to upgrade his hardware.


# Model Fine Tuning

In [33]:
torch_device = torch.device(DEVICE)

## Load Dataset and LLM

In [34]:
hugging_face_dataset_name = "knkarthick/dialogsum"

In [35]:
dataset = load_dataset(hugging_face_dataset_name)

In [36]:
model_name='google/flan-t5-base'

original_model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(torch_device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [37]:
def number_of_trainable_model_parameters(model):
        trainable_model_params = 0
        all_model_params = 0
        for _, param in model.named_parameters():
            all_model_params += param.numel()
            if param.requires_grad:
                trainable_model_params += param.numel()
        result = f"trainable model parameters: {trainable_model_params}\n"
        result += f"all model parameters: {all_model_params}\n"
        result += f"Percentage of model params: {(trainable_model_params/all_model_params)*100}"
        return result

In [38]:
print(number_of_trainable_model_parameters(original_model))

trainable model parameters: 247577856
all model parameters: 247577856
Percentage of model params: 100.0


## Test the Model with Zero Shot Inferencing

In [39]:
index = 200

dialogue = dataset['test'][index]['dialogue']
summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation

{dialogue}

Summary:
"""

inputs = tokenizer(prompt, return_tensors='pt')
output = tokenizer.decode(
    original_model.generate(
        inputs["input_ids"],
        max_new_tokens=200,
    )[0],
    skip_special_tokens=True
)
dash_line = "-".join("" for x in range(100))
print(dash_line)
print(f"Input Prompt:\n{prompt}")
print(dash_line)
print(f"Baseline Human Summary:\n{summary}\n")
print(dash_line)
print(f"Model Generation - Zero Shot: \n{output}")


---------------------------------------------------------------------------------------------------
Input Prompt:

Summarize the following conversation

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

--------------------------------------------------------------------

## Perform Full Fine-Tunning

### Preprocess the Dialog-Summary dataset

Convert the dialog-summary (prompt-response) pairs into explicit instructions for the LLM. Prepend an instruction to the start of the dialog with 'Summarize the following conversation' and the start of the summary with 'Summary as follows'

In [40]:
def tokenize_function(example):
    start_prompt = 'Summarize the following conversation.\n\n'
    end_prompt = '\n\nSummary: '
    prompt = [start_prompt + dialogue + end_prompt for dialogue in example['dialogue']]
    example['input_ids'] = tokenizer(prompt, padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["summary"], padding="max_length", truncation=True, return_tensors="pt").input_ids

    return example

In [41]:
# The dataset actually contains 3 diff splits: train, validation, test
# The tokenize_function code is handling all data accross all splits in batches

tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets = tokenized_datasets.remove_columns(['id', 'topic', 'dialogue', 'summary',])

Map: 100%|██████████| 1500/1500 [00:00<00:00, 1795.14 examples/s]


To save some time, we will subsample the dataset:

In [42]:
tokenized_datasets = tokenized_datasets.filter(lambda example, index: index % 100 == 0, with_indices=True)

Filter: 100%|██████████| 1500/1500 [00:00<00:00, 1512.20 examples/s]


In [43]:
print(f"Shapes of the datasets:")
print(f"Training: {tokenized_datasets['train'].shape}")
print(f"Validation: {tokenized_datasets['validation'].shape}")
print(f"Test: {tokenized_datasets['test'].shape}")

print(tokenized_datasets)

Shapes of the datasets:
Training: (125, 2)
Validation: (5, 2)
Test: (15, 2)
DatasetDict({
    train: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 125
    })
    validation: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 5
    })
    test: Dataset({
        features: ['input_ids', 'labels'],
        num_rows: 15
    })
})


### Fine-Tune the model with the Preprocessed Dataset

Now utilize the built-in Hugging Face Trainer class.

In [44]:
output_dir = f"./dialogue-summary-training-{str(int(time.time()))}"

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=1e-5,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

trainer = Trainer(
    model=original_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
trainer.train()  

In [46]:
instruct_model = AutoModelForSeq2SeqLM.from_pretrained('full/').to(torch_device)
original_model = original_model.to(torch_device)

## Evaluate the Model Qualitatively

In [47]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors='pt').input_ids

original_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_text_output = tokenizer.decode(original_outputs[0], skip_special_tokens=True)

instruct_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
instruct_text_output = tokenizer.decode(instruct_outputs[0], skip_special_tokens=True)

dash_line = "-".join("" for x in range(100))
print(dash_line)
print(f"Input Prompt:\n{prompt}")
print(dash_line)
print(f"Baseline Human Summary:\n{human_baseline_summary}\n")
print(dash_line)
print(f"Original Model Generation - Zero Shot: \n{original_text_output}")
print(dash_line)
print(f"Instruct Model Generation - Fine Tune: \n{instruct_text_output}")

---------------------------------------------------------------------------------------------------
Input Prompt:

Summarize the following conversation

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

--------------------------------------------------------------------

## Evaluate the Model Quantitatively (with ROUGE Metric)

In [48]:
rouge = evaluate.load('rouge')

In [49]:
dialogue = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
instruct_model_summaries = []
for _, dialogue in enumerate(dialogue):
    prompt = f"""
Summarize the following conversation

{dialogue}

Summary:
    """
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
    
    original_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    original_text_output = tokenizer.decode(original_outputs[0], skip_special_tokens=True)
    original_model_summaries.append(original_text_output)

    instruct_outputs = instruct_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    instruct_text_output = tokenizer.decode(instruct_outputs[0], skip_special_tokens=True)
    instruct_model_summaries.append(instruct_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, instruct_model_summaries))

df = pd.DataFrame(zipped_summaries, columns=['human', 'original', 'instruct'])

In [50]:
df

,human,original,instruct
0,Ms. Dawson helps #Person1# to write a memo to ...,Employees are required to use instant messagin...,#Person1# asks Ms. Dawson to take a dictation ...
1,In order to prevent employees from wasting tim...,This memo will be sent to all employees by thi...,#Person1# asks Ms. Dawson to take a dictation ...
2,Ms. Dawson takes a dictation for #Person1# abo...,Employees are required to use the Office of In...,#Person1# asks Ms. Dawson to take a dictation ...
3,#Person2# arrives late because of traffic jam....,People are talking about the traffic in this c...,#Person2# got stuck in traffic again. #Person1...
4,#Person2# decides to follow #Person1#'s sugges...,#Person1: I'm finally here!,#Person2# got stuck in traffic again. #Person1...
5,#Person2# complains to #Person1# about the tra...,#Person1: I'm sorry to hear that you're stuck ...,#Person2# got stuck in traffic again. #Person1...
6,#Person1# tells Kate that Masha and Hero get d...,Masha and Hero are divorced.,#Person1# tells Kate Masha and Hero are gettin...
7,#Person1# tells Kate that Masha and Hero are g...,Masha and Hero are divorced.,#Person1# tells Kate Masha and Hero are gettin...
8,#Person1# and Kate talk about the divorce betw...,#Person1: Masha and Hero are getting a divorce.,#Person1# tells Kate Masha and Hero are gettin...
9,#Person1# and Brian are at the birthday party ...,"#Person1#: Brian, thank you for coming to the ...",Brian's birthday is coming. Brian dances with ...


In [51]:
original_model_results = rouge.compute(
    predictions=original_model_summaries,
    references=human_baseline_summaries,
    use_aggregator=True,
    use_stemmer=True
)

In [52]:
instruct_model_results = rouge.compute(
    predictions=instruct_model_summaries,
    references=human_baseline_summaries[0:len(instruct_model_summaries)],
    use_aggregator=True,
    use_stemmer=True
)

print(f"Original Model: \n{original_model_results}")
print(f"Instruct Model: \n{instruct_model_results}")


Original Model: 
{'rouge1': 0.261052062988671, 'rouge2': 0.08531489481944488, 'rougeL': 0.224821552384684, 'rougeLsum': 0.22788611265447228}
Instruct Model: 
{'rouge1': 0.38857220563277894, 'rouge2': 0.13135692283806472, 'rougeL': 0.28167162470172985, 'rougeLsum': 0.28344342480768214}


# Parameter Efficient Fine Tunning with LoRA

Now lets perform Parameter Efficient Fine-Tunning (PEFT). Opposed to full fine tunning, PEFT is a form of instruction fine-tunnin hat is much more efficient than full fine-tunning - with comparable evaluation results as you will see soon.

PEFT is a generic term that includes Low-Rank Adaptation (LoRA) and prompt tunning (which is not the same as prompt engineering). In most cases when someone says PEFT, they typically mean LoRA, at a very high level allows the user to fine-tune their model using fewer compute resources (in some cases, a single GPU).

## Setup the PEFT/LoRA model for Fine-Tunning

In [53]:
from peft import LoraConfig, get_peft_model, TaskType, PeftModel, PeftConfig

In [54]:
lora_config = LoraConfig(
    r=32,
    lora_alpha=32,
    target_modules=['q','v'],
    lora_dropout=0.05,
    bias='none',
    task_type=TaskType.SEQ_2_SEQ_LM
)

In [55]:
peft_model = get_peft_model(original_model, lora_config)
print(number_of_trainable_model_parameters(peft_model))

trainable model parameters: 3538944
all model parameters: 251116800
Percentage of model params: 1.4092820552029972


In [56]:
output_dir = f"./peft-dialogue-summary-training-{str(int(time.time()))}"

training_args = TrainingArguments(
    auto_find_batch_size=True,
    output_dir=output_dir,
    learning_rate=1e-3,
    num_train_epochs=100,
    weight_decay=0.01,
    logging_steps=1,
    max_steps=1
)

peft_trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation']
)

In [ ]:
peft_trainer.train()

In [58]:
peft_model_base = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

peft_model = PeftModel.from_pretrained(
    peft_model_base,
    "peft/"
).to(torch_device)
original_model = original_model.to(torch_device)

In [59]:
index = 200
dialogue = dataset['test'][index]['dialogue']
human_baseline_summary = dataset['test'][index]['summary']

prompt = f"""
Summarize the following conversation

{dialogue}

Summary:
"""

input_ids = tokenizer(prompt, return_tensors='pt').input_ids

original_outputs = original_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
original_text_output = tokenizer.decode(original_outputs[0], skip_special_tokens=True)

peft_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200, num_beams=1))
peft_text_output = tokenizer.decode(peft_outputs[0], skip_special_tokens=True)

dash_line = "-".join("" for x in range(100))
print(dash_line)
print(f"Input Prompt:\n{prompt}")
print(dash_line)
print(f"Baseline Human Summary:\n{human_baseline_summary}\n")
print(dash_line)
print(f"Original Model Generation - Zero Shot: \n{original_text_output}")
print(dash_line)
print(f"Instruct Model Generation - Zero Shot: \n{peft_text_output}")

---------------------------------------------------------------------------------------------------
Input Prompt:

Summarize the following conversation

#Person1#: Have you considered upgrading your system?
#Person2#: Yes, but I'm not sure what exactly I would need.
#Person1#: You could consider adding a painting program to your software. It would allow you to make up your own flyers and banners for advertising.
#Person2#: That would be a definite bonus.
#Person1#: You might also want to upgrade your hardware because it is pretty outdated now.
#Person2#: How can we do that?
#Person1#: You'd probably need a faster processor, to begin with. And you also need a more powerful hard disc, more memory and a faster modem. Do you have a CD-ROM drive?
#Person2#: No.
#Person1#: Then you might want to add a CD-ROM drive too, because most new software programs are coming out on Cds.
#Person2#: That sounds great. Thanks.

Summary:

--------------------------------------------------------------------

In [60]:
dialogue = dataset['test'][0:10]['dialogue']
human_baseline_summaries = dataset['test'][0:10]['summary']

original_model_summaries = []
peft_model_summaries = []
for _, dialogue in enumerate(dialogue):
    prompt = f"""
Summarize the following conversation

{dialogue}

Summary:
    """
    input_ids = tokenizer(prompt, return_tensors='pt').input_ids
    
    peft_outputs = peft_model.generate(input_ids=input_ids, generation_config=GenerationConfig(max_new_tokens=200))
    peft_text_output = tokenizer.decode(peft_outputs[0], skip_special_tokens=True)
    peft_model_summaries.append(peft_text_output)

zipped_summaries = list(zip(human_baseline_summaries, original_model_summaries, peft_model_summaries))

df = pd.DataFrame(zipped_summaries, columns=['human', 'original', 'peft'])

In [61]:
peft_model_results = rouge.compute(
    predictions=peft_model_summaries,
    references=human_baseline_summaries[0:len(peft_model_summaries)],
    use_aggregator=True,
    use_stemmer=True
)

print(f"Original Model: \n{original_model_results}")
print(f"Instruct Model: \n{instruct_model_results}")
print(f"Peft Model: \n{peft_model_results}")

Original Model: 
{'rouge1': 0.261052062988671, 'rouge2': 0.08531489481944488, 'rougeL': 0.224821552384684, 'rougeLsum': 0.22788611265447228}
Instruct Model: 
{'rouge1': 0.38857220563277894, 'rouge2': 0.13135692283806472, 'rougeL': 0.28167162470172985, 'rougeLsum': 0.28344342480768214}
Peft Model: 
{'rouge1': 0.33176278482581173, 'rouge2': 0.08811333505050914, 'rougeL': 0.2509677309788697, 'rougeLsum': 0.25262149176905513}


# **Knowledge Grounding**

Knowledge grounding in Natural Language Processing (NLP) refers to the process of connecting or linking information in text to real-world entities or concepts. It involves ensuring that the language model understands and can relate the information it processes to factual, contextual, or external knowledge.

For instance, if a sentence mentions "Einstein's theory of relativity," knowledge grounding would involve the model recognizing that Einstein refers to a renowned physicist and the theory of relativity is a fundamental concept in physics.

Knowledge grounding is crucial for NLP applications that require a deeper understanding of the world, as it enables the model to go beyond surface-level patterns in text and make meaningful inferences based on its understanding of the underlying concepts. This is particularly important in tasks like question-answering, where the model needs to provide accurate and contextually relevant responses.
Knowledge grounding with Retrieval Augmented Generation (RAG) is implemented to mitigate hallucinations and provide trustworthy and reliable responses. This is achieved by incorporating information from external sources to validate and support the generated text.



## To know more

### RAG

https://towardsdatascience.com/build-industry-specific-llms-using-retrieval-augmented-generation-af9e98bb6f68

### Langchain

https://python.langchain.com/docs/get_started/introduction

### OpenAI API Key

Note: Please note that the use of the OpenAI API may require the utilization of allocated free credits or additional purchases for implementing the project. Kindly take note of the free credits limit provided for your usage.

https://www.howtogeek.com/885918/how-to-get-an-openai-api-key/#:~:text=Go%20to%20OpenAI's%20Platform%20website,generate%20a%20new%20API%20key.

In [ ]:
!streamlit run llm_app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.27.12.174:8501

  For better performance, install the Watchdog module:

  $ xcode-select --install
  $ pip install watchdog
            
a5255313ec7d762337e1d1f02b6be7c9
Number of requested results 4 is greater than number of elements in index 2, updating n_results = 2


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
System: Use the following pieces of context to answer the users question. 
If you don't know the answer, just say that you don't know, don't try to make up an answer.
----------------
Printable Design for Apparel Products

Available in: AU, CA, EU, NZ, US with minimum quantity per order of 1 piece and maximum of 50 pieces. Printing goes direct to garment printing. Adult T Shirts for Men and Women are available in sizes S, M, L, XL, XXL in white, black, navy, light steel, deep royal, deep red, smoke gr

## **Conclusion**

In today's world, we see language models doing some pretty amazing things. They help businesses understand text on a big scale and make our online experiences better.

This project was all about getting to know these language models inside out. We looked at how they work, how we can use them better, and even gave them a bit of a tune-up. We made them smarter by teaching them how to answer questions and summarize conversations.

We also built a cool shopping chatbot. This bot is smart because it doesn't make up stuff; it gives you real info about products.

With this project, you've learned a lot about these language models, and you're all set to use them for exciting tasks, from writing text to building smart applications.


Start experimenting !!

## **Interview Questions**


* Can you explain the significance of Large Language Models in today's data-driven world?
* What is the main purpose of prompt engineering in the context of LLMs?
* How does fine-tuning enhance the performance of a pre-trained language model?
* What is the difference between full fine-tuning and Parameter Efficient Fine Tuning (PEFT)?
* Can you explain the concept of Retrieval Augmented Generation (RAG) and how it was implemented in the project?
* How does knowledge grounding improve the reliability of responses generated by the chatbot?
* What is Reinforcement Learning from Human Feedback (RLHF)?

* How did you evaluate the performance of the models in this project, and what metrics did you use?
How did you handle the potential issue of hallucinations in the chatbot's responses?
* What challenges did you encounter during the implementation of fine-tuning techniques, and how did you overcome them?
* How would you address potential ethical concerns related to the use of language models in real-world applications?
* What are some potential future improvements or extensions you would consider for this project?